<a href="https://colab.research.google.com/github/MichaelTellis/COSMOS_SPARC_project/blob/main/Density_as_a_function_of_Radius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Don't forget to run the first block of code below!

In [13]:
import os
import numpy as np
import pandas as pd


#Upload the summary file and name it as: galaxy_sample_summary.txt
#Also have everyone single SPARC data uploaded

# Constants
GNewton = 4.43e-6  # kpc (km/s)^2 / solar mass

# Summary file and column names
SUMMARY_FILE = "galaxy_sample_summary.txt"
COLUMN_NAMES = ["Galaxy Name", "Hubble Type (1)", "Distance Mpc", "Mean error on D Mpc",
"Distance Method (2)", "Inclination deg", "Mean error on Inc deg", "Total Luminosity at [3.6]_10+9solLum",
"Mean error on L[3.6]_10+9solLum", "Effective Radius at [3.6] kpc", "Effective Surface Brightness at [3.6]_solLum/pc2",
"Disk Scale Length at [3.6]_kpc", "Disk Central Surface Brightness at [3.6]_solLum/pc2", "Total HI mass_10+9solMass",
"HI radius at 1 Msun/pc2_kpc", "Asymptotically Flat Rotation Velocity km/s", "Mean error on Vflat km/s", "Quality Flag (3)",
"References for HI and Ha data (4)"]
COLSPECS = [(0, 11), (12, 14), (14, 20), (20, 25), (25, 27), (27, 31), (31, 35), (35, 42), (42, 49), (49, 54), (54, 62), (62, 67), (67, 75), (75, 82), (82, 87), (87, 92), (92, 97), (97, 100), (100, 114)]

# Read the summary file
summary_df = pd.read_fwf(SUMMARY_FILE, header=None, names=COLUMN_NAMES, colspecs=COLSPECS)
galaxy_names = summary_df[COLUMN_NAMES[0]].tolist()

# Function to calculate densities
def calculate_densities(R, V_obs, V_visible):
    V_shell = [(4./3) * np.pi * R[0]**3]
    M_shell = [R[0] * V_obs[0]**2 / GNewton]
    M_vis_shell = [R[0] * V_visible[0]**2 / GNewton]
    R_midpt = [0.5 * R[0]]

    for i in range(1, len(R)):
        V_shell.append((4./3) * np.pi * (R[i]**3 - R[i-1]**3))
        M_shell.append(R[i] * V_obs[i]**2 / GNewton - M_shell[i-1])
        M_vis_shell.append(R[i] * V_visible[i]**2 / GNewton - M_vis_shell[i-1])
        R_midpt.append(0.5 * (R[i] + R[i-1]))

    density = np.array(M_shell) / np.array(V_shell)
    vis_density = np.array(M_vis_shell) / np.array(V_shell)
    dark_matter_density = density - vis_density

    return R_midpt, density, vis_density, dark_matter_density

# Initialize dictionary to store formatted data
formatted_data = {}

# Loop over each galaxy
for galaxy_name in galaxy_names:
    galaxy_file = galaxy_name + "_rotmod.dat"
    if os.path.isfile(galaxy_file):  # Check if the file exists
        R, V_obs, error_V_obs, V_gas, V_disk, V_bulge, SB_disk, SB_bulge = np.loadtxt(galaxy_file, unpack=True)

        V_visible = np.sqrt(V_gas**2 + V_disk**2 + V_bulge**2)
        R_midpt, density, vis_density, dark_matter_density = calculate_densities(R, V_obs, V_visible)

        # Store formatted data
        formatted_data[f"{galaxy_name}_Radius"] = R_midpt
        formatted_data[f"{galaxy_name}_Total_Mass_Density"] = density
        formatted_data[f"{galaxy_name}_Visible_Mass_Density"] = vis_density
        formatted_data[f"{galaxy_name}_Dark_Matter_Density"] = dark_matter_density

        print(f"Galaxy name: {galaxy_name}")
        print(f"{galaxy_name}_Radius = {R_midpt}")
        print(f"{galaxy_name}_Total_Mass_Density = {density}")
        print(f"{galaxy_name}_Visible_Mass_Density = {vis_density}")
        print(f"{galaxy_name}_Dark_Matter_Density = {dark_matter_density}")
    else:
        print(f"{galaxy_file} not found, skipping this galaxy.")

# Print all the formatted data
for key, value in formatted_data.items():
    print(f"{key} = {value}")

# Optional: Convert the formatted data to a pandas DataFrame for better organization
formatted_df = pd.DataFrame.from_dict(formatted_data, orient='index').transpose()

# Optional: Save the DataFrame to a TSV file for further analysis
formatted_df.to_csv("galaxy_densities.tsv", sep='\t', index=False)


Streaming output truncated to the last 5000 lines.
 7.97032697e+07 6.61120020e+07 5.51703657e+07 5.21926805e+07
 4.34219524e+07 4.18394873e+07 3.43964980e+07 3.30087223e+07
 2.79519952e+07]
NGC3877_Visible_Mass_Density = [7.38859347e+08 1.63382672e+08 1.07129704e+08 7.95826830e+07
 7.22349629e+07 7.25353799e+07 7.84723903e+07 6.89678627e+07
 6.36040814e+07 5.68565670e+07 5.10002771e+07 4.05988496e+07
 3.78685557e+07]
NGC3877_Dark_Matter_Density = [-6.37128477e+08 -4.73989349e+07  1.47310021e+07  1.12132164e+07
  7.46830687e+06 -6.42337793e+06 -2.33020247e+07 -1.67751822e+07
 -2.01821290e+07 -1.50170797e+07 -1.66037790e+07 -7.59012732e+06
 -9.91656043e+06]
Galaxy name: NGC3893
NGC3893_Radius = [0.87, 2.62, 4.37, 6.11, 7.8950000000000005, 9.855, 12.01, 14.225, 16.315, 18.175]
NGC3893_Total_Mass_Density = [3.74235664e+08 9.94235340e+07 6.82036179e+07 3.41011306e+07
 2.83212110e+07 1.80393976e+07 1.42188742e+07 9.50098959e+06
 1.02263270e+07 7.21614257e+06]
NGC3893_Visible_Mass_Density = [